In [2]:
from transformers import pipeline
import pandas as pd

In [4]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification


In [3]:
df = pd.read_csv("../src/article_scraping/data/articles_with_text.csv")
df

,url,titles,descriptions,publishing_date,source,article_text_raw
0,https://www.aljazeera.com/program/between-us/2...,Trump’s road to the election,What is it like covering the Republican campaign?,2024-11-04T08:36:15Z,Al Jazeera English,What is it like covering the Republican campai...
1,https://www.aljazeera.com/program/between-us/2...,Harris’s road to the election,What is it like covering the Democratic campaign?,2024-11-04T08:48:07Z,Al Jazeera English,What is it like covering the Democratic campai...
2,https://www.aljazeera.com/news/2024/11/12/mold...,Moldova formally protests alleged Russian elec...,Chisinau accuses Moscow of undermining democra...,2024-11-12T13:25:32Z,Al Jazeera English,Chisinau accuses Moscow of undermining democra...
3,https://www.aljazeera.com/news/2024/11/12/oppo...,Opposition secures landslide victory in Maurit...,Former Prime Minister Navin Ramgoolam has won ...,2024-11-12T15:44:15Z,Al Jazeera English,Former prime minister Navin Ramgoolam has won ...
4,https://www.aljazeera.com/program/now-you-know...,Will abortion access decide the US election?,The US faces a pivotal election on November 5.,2024-11-04T12:28:07Z,Al Jazeera English,The US faces a pivotal election on November 5....
...,...,...,...,...,...,...
3680,https://apnews.com/article/lewiston-maine-mass...,"One year after a massacre in Maine, survivors ...",NaN,NaN,AP News,"LEWISTON, Maine (AP) — Ben Dyer was shot five ..."
3681,https://apnews.com/article/harris-trump-progre...,Progressives warn Harris must change her closi...,NaN,NaN,AP News,NEW YORK (AP) — Progressive Democrats warn Kam...
3682,https://apnews.com/article/misinformation-trum...,"To fight misinfo, news outlets are putting pla...",NaN,NaN,AP News,Follow AP’s coverage of the election and what ...
3683,https://apnews.com/article/kansas-nebraska-pub...,Conservatives could win control of Kansas and ...,NaN,NaN,AP News,Follow AP’s coverage of the election and what ...


In [ ]:
import re

def clean_text(text, 
               lowercase=True, 
               remove_punctuation=True):

    # Convert to lowercase
    if lowercase:
        text = text.lower()

    # Remove punctuation
    if remove_punctuation:
        text = re.sub(r'[^\w\s]', '', text)

    # Remove extra whitespace (including newlines, tabs)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [37]:
pipe1 = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", truncation=True)     

/home/nadavgerner/anaconda3/envs/dsan5400/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [38]:
pipe2 = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", truncation=True)     

In [45]:
clean_texts = df['article_text_raw'].apply(clean_text)

for i in range(10):
    print('\n', i)
    try:
        text = clean_texts.iloc[i]
        print("distilroberta:", pipe1(text))
        print("nlptown:", pipe2(text))
    except Exception as e:
        print(f'caught {e}')


 0
distilroberta: [{'label': 'neutral', 'score': 0.9998708963394165}]
nlptown: [{'label': '4 stars', 'score': 0.3148525059223175}]

 1
distilroberta: [{'label': 'neutral', 'score': 0.9998527765274048}]
nlptown: [{'label': '4 stars', 'score': 0.30847808718681335}]

 2
distilroberta: [{'label': 'neutral', 'score': 0.9451748132705688}]
nlptown: [{'label': '1 star', 'score': 0.6585333347320557}]

 3
distilroberta: [{'label': 'positive', 'score': 0.9995077848434448}]
nlptown: [{'label': '1 star', 'score': 0.43168556690216064}]

 4
distilroberta: [{'label': 'neutral', 'score': 0.9998759031295776}]
nlptown: [{'label': '4 stars', 'score': 0.416075736284256}]

 5
distilroberta: [{'label': 'negative', 'score': 0.9915562868118286}]
nlptown: [{'label': '2 stars', 'score': 0.37488558888435364}]

 6
distilroberta: [{'label': 'neutral', 'score': 0.9998801946640015}]
nlptown: [{'label': '4 stars', 'score': 0.49186262488365173}]

 7
distilroberta: [{'label': 'neutral', 'score': 0.5001826882362366}]
nl

* Project Demo

using the index from the project demo

In [53]:
similar_indices = [1105, 30, 772, 3181, 96]

for index in similar_indices:
    title = df.iloc[index]['titles']
    source = df.iloc[index]['source']
    text = clean_texts.iloc[index]
    sentiment = pipe2(text)[0]
    label = sentiment['label']

    print(F'Title: {title}')
    print(F'Source: {source}')
    print(F'Score: {label}\n')


Title: Post-election decision: Do you want to live in a red state or a blue state?
Source: Fox News
Score: 2 stars

Title: A simple guide to the US election 2024
Source: Al Jazeera English
Score: 2 stars

Title: Election anxiety leads many Americans to avoid family and friends, survey finds
Source: Fox News
Score: 2 stars

Title: The world watches as US election results trickle in
Source: AP News
Score: 2 stars

Title: Harris, Trump hit North Carolina in final weekend push before US election
Source: Al Jazeera English
Score: 4 stars

